In [44]:
import os
import sys
import shutil
import argparse
import traceback
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc
from statistics import median

# ******************************* User Settings *******************************
PKT_LEN = 250  # Bytes
DATA_RATE = 1000e3  # bits-per-second
PKT_RATE = DATA_RATE / PKT_LEN / 8  # packets-per-second
print("packet_rate (pps):", PKT_RATE, "\n")
# *****************************************************************************

def truncate(txdf, rxdf):
    """
    Truncate the rows of lost packets.
    """
    tx_arr = list(zip(txdf['sequence.number'].array, txdf['Timestamp'].array, txdf['Timestamp_epoch'].array))
    rx_arr = list(zip(rxdf['sequence.number'].array, rxdf['Timestamp'].array, rxdf['Timestamp_epoch'].array))
    _tx_arr = []
    j = 0
    N = len(rx_arr)
    for i in range(len(tx_arr)):
        if tx_arr[i][0] == rx_arr[j][0]:
            _tx_arr.append(tx_arr[i])
            if j < N-1:
                j += 1
    M = min(len(_tx_arr), len(rx_arr))
    return _tx_arr[:M], rx_arr[:M]

def calc_delta(txdl_df, rxdl_df, txul_df, rxul_df):
    """
    Calculate the time delta between server and client.
        If the client is behind server, delta > 0
        If the client is ahead of server, delta < 0
        server clock := client clock + delta
        
    Returns:
        delta (datetime.timedelta)
        delta (float)
    """
    ### Since the transmission is launched by client, the starting time of Uplink is ahead of Downlink.
    seq_diff = round(500e-3 * PKT_RATE)
    txul_df = txul_df[txul_df['sequence.number'] > seq_diff].reset_index(drop=True)
    rxul_df = rxul_df[rxul_df['sequence.number'] > seq_diff].reset_index(drop=True)
    
    txdl_arr, rxdl_arr = truncate(txdl_df, rxdl_df)
    txul_arr, rxul_arr = truncate(txul_df, rxul_df)

    M = min(len(txdl_arr), len(txul_arr))
    txdl_arr, rxdl_arr, txul_arr, rxul_arr = txdl_arr[:M], rxdl_arr[:M], txul_arr[:M], rxul_arr[:M]

    # print(len(txdl_arr), len(rxdl_arr))
    # print(txdl_arr[0], rxdl_arr[0])
    # print(txdl_arr[-1], rxdl_arr[-1])
    # print()
    # print(len(txul_arr), len(rxul_arr))
    # print(txul_arr[0], rxul_arr[0])
    # print(txul_arr[-1], rxul_arr[-1])
    # print()
    
    timedelta_list = []
    epoch_delta_list = []
    for ts1, ts2, ts3, ts4 in zip(txdl_arr, rxdl_arr, txul_arr, rxul_arr):
        latency_dl = ts2[1] - ts1[1]
        epoch_latency_dl = ts2[2] - ts1[2]
        latency_ul = ts4[1] - ts3[1]
        epoch_latency_ul = ts4[2] - ts3[2]
        timedelta_list.append((latency_ul - latency_dl) / 2)
        epoch_delta_list.append((epoch_latency_ul - epoch_latency_dl) / 2)
    return median(timedelta_list), round(median(epoch_delta_list), 6)

packet_rate (pps): 500.0 



In [45]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:00.153820500
0.15382


In [46]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:01.193984500
1.193985


In [47]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:01.074737500
1.074738


In [48]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:00.674709500
0.674709


In [49]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm07/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm07/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm07/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm07/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:00.548389
0.548389


In [50]:
txdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm08/#01/analysis/srv_dwnlnk_udp_packet_brief.csv")
rxdl_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm08/#01/analysis/clt_dwnlnk_udp_packet_brief.csv")
txul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm08/#01/analysis/clt_uplnk_udp_packet_brief.csv")
rxul_df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm08/#01/analysis/srv_uplnk_udp_packet_brief.csv")

txdl_df['Timestamp'] = pd.to_datetime(txdl_df['Timestamp'])  # transmitted time from server
rxdl_df['Timestamp'] = pd.to_datetime(rxdl_df['Timestamp'])  # arrival time to client
txul_df['Timestamp'] = pd.to_datetime(txul_df['Timestamp'])  # transmitted time from client
rxul_df['Timestamp'] = pd.to_datetime(rxul_df['Timestamp'])  # arrival time to server

delta, epoch_delta = calc_delta(txdl_df, rxdl_df, txul_df, rxul_df)
print(delta)
print(epoch_delta)

0 days 00:00:00.988643
0.988643
